In [1]:
import torch.optim as optim

import bayescache.api as api
from bayescache.models import mtcnn
from bayescache.data.anon import Anon
from bayescache.api.source import TrainingData
from bayescache.callbacks.time_tracker import TimeTracker

In [2]:
model = mtcnn.new()

optimizer = optim.RMSprop(model.parameters(), lr=7.0e-4, eps=1e-3)

In [3]:
learner = api.Learner(device='cpu', model=model)
learner.summary()

MTCNN(
  (embedding): Embedding(
    (emb): Sequential(
      (embedding_35095_300): Embedding(35095, 300, padding_idx=0)
    )
  )
  (conv1): Conv1d(
    (conv): Sequential(
      (conv1d_3_300): Conv1d(1, 300, kernel_size=(3,), stride=(1,))
    )
  )
  (conv2): Conv1d(
    (conv): Sequential(
      (conv1d_4_300): Conv1d(1, 300, kernel_size=(4,), stride=(1,))
    )
  )
  (conv3): Conv1d(
    (conv): Sequential(
      (conv1d_5_300): Conv1d(1, 300, kernel_size=(5,), stride=(1,))
    )
  )
  (fc1): Linear(in_features=900, out_features=34, bias=True)
  (fc2): Linear(in_features=900, out_features=4, bias=True)
  (fc3): Linear(in_features=900, out_features=3, bias=True)
  (fc4): Linear(in_features=900, out_features=44, bias=True)
  (fc5): Linear(in_features=900, out_features=5, bias=True)
)
--------------------------------------------------------------------------------
Number of model parameters: 10,614,090
--------------------------------------------------------------------------------


In [4]:
train = Anon(root='/Users/yngtodd/data', partition='train', download=True)
test = Anon(root='/Users/yngtodd/data', partition='test', download=True)

source = TrainingData(train_source=train, val_source=test, num_workers=2, batch_size=1)

In [5]:
metrics = learner.metrics()
callbacks = [TimeTracker()]

training_info = api.TrainingInfo(
    start_epoch_idx=0,
    run_name='test',
    metrics=metrics,
    callbacks=callbacks
)

In [6]:
training_info.on_train_begin()

In [7]:
for global_epoch_idx in range(training_info.start_epoch_idx + 1, 1 + 1):
    epoch_info = api.EpochInfo(
        training_info=training_info,
        global_epoch_idx=global_epoch_idx,
        batches_per_epoch=source.train_iterations_per_epoch(),
        optimizer=optimizer
    )

    # Execute learning
    learner.run_epoch(epoch_info, source)

|-------- Epoch 000001 Lr=0.000700 ----------|
Training:   0%|          | 0/2500 [00:00<?, ?iter/s]

RuntimeError: dimension specified as 0 but tensor has no dimensions